<a href="https://colab.research.google.com/github/carlibeisel/Drains_Lower_Boise_River/blob/main/03_climate_data_extract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Modified by Carli Beisel

Adapted from code written by Bridget Bittmann (2023, Github: bridgetmarie24)

Date originally created: July 20, 2022

Date modified: May 13, 2024

Purpose: Calculates annual zonal stats for each drain watershed from Daymet and SSEBop

In [4]:
# Installs geemap package
import subprocess
!pip install geemap

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project = 'extract-gridmet')




     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.7 MB/s eta 0:00:00


In [1]:
!pip install geopandas
import geopandas as gpd #import independent shapefile
import json #for metadata of shapefile
import os #for file paths
import numpy as np #for stats and arrays
import pandas as pd #for dataframes
!pip install pycrs
import pycrs

  Preparing metadata (setup.py) ... done
  Created wheel for pycrs: filename=PyCRS-1.0.2-py3-none-any.whl size=32687 sha256=d26e6f11a46380d00a3b980024ad3119c5f92835dc6fc0a3a22854b0f9863f56
  Stored in directory: /root/.cache/pip/wheels/47/1d/70/7a5bdf33347e7c75e95b06b1fa38f076a59a9506653cc24aff
Successfully built pycrs


In [2]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
## ------------------------------------------------------- ##
## 1. Import shapefile and start/end dates to clip dataset ##
## ------------------------------------------------------- ##
start_end = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_input/start_end_dates.csv')
shp_file = '/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_input/drain_delineation/Drains_Merge_07072022.shp'
subset = emap.shp_to_ee(shp_file) # converts shapefile to feature in GEE

map = emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(subset, 0, 2), {}, 'POU') #problem with POU
map.addLayerControl()
map

Map(center=[43.615, -116.2023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [6]:
## ------------------------------------------------- ##
## IMPORT PRECIP, TEMP, and ET FOR IRRIGATION SEASON ##
## ------------------------------------------------- ##

#changing to account for full annual precip - not just irrigation season.
#all old code is # out above the changes

years = np.arange(1987,2021)

# Empty lists to store images
ir_tmp = []
#ir_pr = []
annual_pr = []
mean_max = []
et_irrig = []

# Import image collection, subset to shapefile, take a statistics for a period of time,
# and append the image to the designated list
for i in range(len(start_end)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate(start_end['StartDate'][i], start_end['EndDate'][i]) #get image collection for irrigation season
  daymet_hot = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-06-01'), (str(years[i])+'-8-31')) #get image collection for June-Aug
  et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(years[i])+'-04-01'), str(years[i])+'-10-31')
  et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(years[i])}) # sum et and convert to meters
  mxtmp = daymet_hot.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index':str(years[i])}) #select temp to analyze hot months and take mean
  tmp = daymet.select('tmax').map(lambda image: image.clip(subset)).mean().set({'system:index':str(start_end['StartDate'][i])}) #select max temp to analyze and take mean
  #pr = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index': str(start_end['StartDate'][i])}) #select precip to analyze and sum
  pr = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index': str(start_end['Year'][i])}) #select precip to analyze and sum
  ir_tmp.append(tmp)
  #ir_pr.append(pr)
  annual_pr.append(pr)
  mean_max.append(mxtmp)
  et_irrig.append(et)

# Convert lists of images to image collection for zonal stats command
et_irrig = ee.ImageCollection(et_irrig)
ir_tmp = ee.ImageCollection(ir_tmp)
#ir_pr = ee.ImageCollection(ir_pr)
annual_pr = ee.ImageCollection(annual_pr)
means_max_temp = ee.ImageCollection(mean_max)

In [14]:
## ---------------------------------------------------------------------- ##
## 3. IMPORT THE DAYMET DATA FOR PRECIPITATION PRIOR TO IRRIGATION SEASON ##
## ---------------------------------------------------------------------- ##

# This will provide insight into antecedent moisture conditions for a POU.
#I changed this code so that it collects the full year antecedent precipitation - not just within the irrigation season

years = np.arange(1987,2021)
ant_pr = []
for i in range(len(years)):
  #daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i]-1)+'-10-31'), start_end['StartDate'][i]) #get image collection
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate(str(years[i] - 1) + '-01-01', str(years[i] - 1) + '-12-31') #trying to get full year
  date = str(years[i]+1)
  print(date)
  prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set({'system:index':date}) #select the bands to analyze
  ant_pr.append(prcp) #calculate the mean across all pixels

ant_precip = ee.ImageCollection(ant_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(ant_precip, precip_vis, 'prcp')
Map

1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


Map(center=[43.615, -116.2023], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchD…

In [17]:
## ------------------------ ##
## 4. CALCULATE ZONAL STATS ##
## ------------------------ ##

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

out_stats = os.path.join('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/JAtemp_stats.csv')
emap.zonal_statistics(means_max_temp, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/ir_tmp_stats.csv')
emap.zonal_statistics(ir_tmp, subset, out_stats, statistics_type='MEAN', scale=1000)

#old for irrigation precip
#out_stats = os.path.join('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/irrig_precip_stats.csv')
#emap.zonal_statistics(ir_pr, subset, out_stats, statistics_type='MEAN', scale=1000)

#new for annual precip
out_stats = os.path.join('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/annual_precip_stats.csv')
emap.zonal_statistics(annual_pr, subset, out_stats, statistics_type='MEAN', scale=1000)

#this will also be new because of the year-1 change . . .
out_stats = os.path.join('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/ant_precip_stats.csv')
emap.zonal_statistics(ant_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

out_stats = os.path.join('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/et.csv')
emap.zonal_statistics(et_irrig, subset, out_stats, statistics_type='MEAN', scale=30)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/JAtemp_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/ir_tmp_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/annual_precip_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/ant_precip_stats.csv
Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/et.csv


In [18]:
## ---------------------------------------------- ##
## 5. CREATE CLIMATE STAT FOR EACH POU AND EXPORT ##
## ---------------------------------------------- ##

years = np.arange(1987,2021)
#old
#ir_precip = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/irrig_precip_stats.csv')
#new
annual_precip = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/annual_precip_stats.csv')
ant_precip = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/ant_precip_stats.csv')
JA_temp = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/JAtemp_stats.csv')
irrig_temp = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/ir_tmp_stats.csv')
et_irrig = pd.read_csv('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/et.csv')


names = et_irrig['Name']

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['NAME'] = names[i]
  df['ant_prcp'] = ant_precip.iloc[i,0:34].values
  #old
  #df['irrig_prcp'] = ir_precip.iloc[i,0:34].values
  #new
  df['annual_prcp'] = annual_precip.iloc[i,0:34].values
  df['irrig_temp'] = irrig_temp.iloc[i,0:34].values
  df['JuneAug_temp'] = JA_temp.iloc[i,0:34].values
  df['et'] = et_irrig.iloc[i,0:34].values
  out_path = os.path.join('/content/drive/MyDrive/Data/Drains_Lower_Boise_River/data_output/climate_data_extract_out/final/'+names[i]+'_climate_0531.csv')
  df.to_csv(out_path)
